# 🤖 Agentic AI Scheduling Assistant Demo

This notebook demonstrates the complete Agentic AI Scheduling Assistant implementation for the AMD Hackathon.

## Features:
- ✅ Autonomous email parsing with LLM
- ✅ Intelligent meeting scheduling
- ✅ Conflict detection and resolution
- ✅ Fast response time (<10 seconds)
- ✅ Proper JSON output format

## 1. Import Dependencies and Initialize

In [4]:
import json
import requests
from datetime import datetime, timedelta
import re
from typing import Dict, List, Tuple, Optional
from dateutil import parser as date_parser
import pytz
import time

print("📦 Dependencies imported successfully!")

📦 Dependencies imported successfully!


## 2. Load Sample Input Data

In [5]:
# Load the sample input JSON
with open('JSON_Samples/Input_Request.json', 'r') as f:
    sample_input = json.load(f)

print("📧 Sample Input Request:")
print(json.dumps(sample_input, indent=2))

📧 Sample Input Request:
{
  "Request_id": "6118b54f-907b-4451-8d48-dd13d76033a5",
  "Datetime": "19-07-2025T12:34:55",
  "Location": "IISc Bangalore",
  "From": "userone.amd@gmail.com",
  "Attendees": [
    {
      "email": "usertwo.amd@gmail.com"
    },
    {
      "email": "userthree.amd@gmail.com"
    }
  ],
  "Subject": "Agentic AI Project Status Update",
  "EmailContent": "Hi team, let's meet on Thursday for 30 minutes to discuss the status of Agentic AI Project."
}


## 3. Import Our Agentic Scheduler

In [6]:
from agentic_scheduler import AgenticScheduler

# Initialize the scheduler
scheduler = AgenticScheduler()
print("🤖 Agentic Scheduler initialized!")

ModuleNotFoundError: No module named 'agentic_scheduler'

## 4. Demonstrate Email Parsing

In [ ]:
# Test the email parsing functionality
email_content = sample_input['EmailContent']
subject = sample_input['Subject']
attendees = [sample_input['From']] + [a['email'] for a in sample_input['Attendees']]

print("🧠 Parsing email content with AI...")
parsed_info = scheduler.parse_email_content(email_content, subject, attendees)

print("\n📋 Extracted Information:")
print(json.dumps(parsed_info, indent=2))

## 5. Show Calendar Event Retrieval

In [ ]:
# Demonstrate calendar event retrieval for each attendee
print("📅 Retrieving calendar events for all attendees:")

for attendee in attendees:
    events = scheduler.get_calendar_events(attendee, "2025-07-24", "2025-07-25")
    print(f"\n👤 {attendee}:")
    if events:
        for event in events:
            print(f"  • {event['Summary']}: {event['StartTime']} - {event['EndTime']}")
    else:
        print("  • No existing events")

## 6. Find Optimal Meeting Slot

In [ ]:
# Find the optimal meeting slot
print("🎯 Finding optimal meeting slot...")

start_time, end_time = scheduler.find_optimal_slot(
    attendees,
    parsed_info.get('meeting_duration_minutes', 30),
    parsed_info.get('time_preference', 'thursday'),
    sample_input['Datetime']
)

print(f"\n⏰ Optimal Meeting Slot:")
print(f"Start: {start_time}")
print(f"End: {end_time}")
print(f"Duration: {parsed_info.get('meeting_duration_minutes', 30)} minutes")

## 7. Complete Processing Demo

In [ ]:
# Process the complete meeting request
print("🚀 Processing complete meeting request...")

start_time = time.time()
result = scheduler.process_meeting_request(sample_input)
end_time = time.time()

processing_time = end_time - start_time

print(f"\n⏱️ Processing Time: {processing_time:.3f} seconds")
print(f"✅ Latency Requirement: {'PASSED' if processing_time < 10 else 'FAILED'} (<10 sec)")

## 8. Display Final Output

In [ ]:
print("📄 Final Scheduled Output:")
print("=" * 60)
print(json.dumps(result, indent=2))

## 9. Validation and Scoring

In [ ]:
# Validate the output format
print("🔍 Validating Output Format:")
print("=" * 40)

required_fields = [
    'Request_id', 'Datetime', 'Location', 'From', 
    'Attendees', 'Subject', 'EmailContent', 
    'EventStart', 'EventEnd', 'Duration_mins', 'MetaData'
]

score = 0
total_checks = len(required_fields) + 4  # Additional checks

# Check required fields
for field in required_fields:
    if field in result:
        print(f"✅ {field}: Present")
        score += 1
    else:
        print(f"❌ {field}: Missing")

# Check attendee structure
if isinstance(result.get('Attendees'), list) and len(result['Attendees']) > 0:
    attendee_valid = True
    for attendee in result['Attendees']:
        if not isinstance(attendee, dict) or 'email' not in attendee or 'events' not in attendee:
            attendee_valid = False
            break
    
    if attendee_valid:
        print("✅ Attendee Structure: Valid")
        score += 1
    else:
        print("❌ Attendee Structure: Invalid")
else:
    print("❌ Attendee Structure: Missing")

# Check time format
try:
    date_parser.parse(result['EventStart'])
    date_parser.parse(result['EventEnd'])
    print("✅ Time Format: Valid ISO format")
    score += 1
except:
    print("❌ Time Format: Invalid")

# Check duration
if result.get('Duration_mins', '').isdigit():
    print("✅ Duration: Valid numeric format")
    score += 1
else:
    print("❌ Duration: Invalid format")

# Check latency
if processing_time < 10.0:
    print("✅ Latency: Under 10 seconds")
    score += 1
else:
    print("❌ Latency: Over 10 seconds")

print("\n" + "=" * 40)
print(f"📊 Overall Score: {score}/{total_checks} ({score/total_checks*100:.1f}%)")

if score == total_checks:
    print("🏆 EXCELLENT! Ready for submission!")
elif score >= total_checks * 0.8:
    print("✅ GOOD! Minor issues to fix.")
else:
    print("⚠️ NEEDS WORK! Major issues detected.")

## 10. Performance Benchmark

In [ ]:
# Run performance benchmark
print("⚡ Performance Benchmark:")
print("=" * 30)

times = []
for i in range(5):
    start = time.time()
    _ = scheduler.process_meeting_request(sample_input)
    end = time.time()
    
    duration = end - start
    times.append(duration)
    print(f"Run {i+1}: {duration:.3f}s")

avg_time = sum(times) / len(times)
min_time = min(times)
max_time = max(times)

print(f"\n📈 Performance Summary:")
print(f"Average: {avg_time:.3f}s")
print(f"Fastest: {min_time:.3f}s")
print(f"Slowest: {max_time:.3f}s")
print(f"Requirement: <10.0s")
print(f"Status: {'✅ PASSED' if avg_time < 10 else '❌ FAILED'}")

## 🎯 Summary

This Agentic AI Scheduling Assistant demonstrates:

### ✅ Key Features:
- **Autonomous Coordination**: AI parses emails and schedules independently
- **Dynamic Adaptability**: Handles time preferences and conflicts
- **Natural Language Processing**: Understands meeting requirements
- **Fast Response**: Optimized for sub-10 second performance

### 🏆 Hackathon Readiness:
- Correct JSON output format
- Proper error handling
- Performance optimization
- Clean, documented code

### 🚀 Next Steps:
1. Start vLLM server
2. Run Submission.ipynb
3. Test external API calls
4. Submit for evaluation!

**Good luck with the hackathon!** 🎉